In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/multi input/multi_input/dataset/ecommerce_sample.csv')

In [2]:
x = df['description']
x

,description
0,Key Features of Elegance Polyester Multicolor ...
1,Specifications of Sathiyas Cotton Bath Towel (...
2,Key Features of Eurospa Cotton Terry Face Towe...
3,Key Features of SANTOSH ROYAL FASHION Cotton P...
4,Key Features of Jaipur Print Cotton Floral Kin...
...,...
1045,Oren Empower Extra Large Self Adhesive Sticker...
1046,Wallmantra Large Vinyl Sticker Sticker (Pack o...
1047,Buy Uberlyfe Extra Large Pigmented Polyvinyl F...
1048,Buy Wallmantra Medium Vinyl Sticker Sticker fo...


In [3]:
df['main_category'] = df['product_category_tree'].apply(lambda x: eval(x)[0].split(' >> ')[0] if pd.notna(x) else None)

In [4]:
y = df['main_category']
y

,main_category
0,Home Furnishing
1,Baby Care
2,Baby Care
3,Home Furnishing
4,Home Furnishing
...,...
1045,Baby Care
1046,Baby Care
1047,Baby Care
1048,Baby Care


In [5]:
!pip install transformers
!pip install keras_nlp

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# encoder les catégories
le = LabelEncoder()
y_encoded = le.fit_transform(y)



In [7]:
import keras_nlp
from tensorflow import keras
import keras_hub


preprocessor = keras_hub.models.DistilBertTextClassifierPreprocessor.from_preset(
    "distil_bert_base_en_uncased",
    sequence_length=128,
)

# Pretrained classifier.
classifier = keras_hub.models.DistilBertTextClassifier.from_preset(
    "distil_bert_base_en_uncased",
    num_classes=7,
    preprocessor=preprocessor,
)
classifier.fit(x=x, y=y_encoded, batch_size=2)



525/525 ━━━━━━━━━━━━━━━━━━━━ 70s 47ms/step - loss: 1.6945 - sparse_categorical_accuracy: 0.3664


In [8]:
df['main_category'].unique()

array(['Home Furnishing', 'Baby Care', 'Watches',
       'Home Decor & Festive Needs', 'Kitchen & Dining',
       'Beauty and Personal Care', 'Computers'], dtype=object)

In [9]:
classifier.predict(["Revitalize your skin with this lightweight, hydrating face serum. Packed with hyaluronic acid and nourishing botanicals, it deeply moisturizes, leaving your skin plump and radiant. Perfect for all skin types, it absorbs quickly and layers well under makeup."])



1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


array([[ 0.40828633,  0.04346664,  0.43931478, -0.27409804,  0.3173388 ,
        -0.5093026 , -1.3987848 ]], dtype=float32)

In [10]:
classifier.layers

[<InputLayer name=padding_mask, built=True>,
 <InputLayer name=token_ids, built=True>,
 <DistilBertBackbone name=distil_bert_backbone, built=True>,
 <Dense name=pooled_dense, built=True>,
 <Dropout name=output_dropout, built=True>,
 <Dense name=logits, built=True>]

In [11]:
for layer in classifier.layers:
    # Vérifie si l'objet a l'attribut 'output_shape' avant de l'afficher
    if hasattr(layer, 'output_shape'):
        print(layer.name, layer.output_shape)
    else:
        print(layer.name, "n'a pas d'attribut 'output_shape'")



padding_mask n'a pas d'attribut 'output_shape'
token_ids n'a pas d'attribut 'output_shape'
distil_bert_backbone (None, None, 768)
pooled_dense n'a pas d'attribut 'output_shape'
output_dropout n'a pas d'attribut 'output_shape'
logits n'a pas d'attribut 'output_shape'


In [31]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
import os
from tensorflow.keras.optimizers import Adam

# Fixer le random seed pour assurer la reproductibilité
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Chemin vers le dossier d'images
images_dir = '/content/drive/MyDrive/multi input/multi_input/dataset/Images_224'

# Liste de tous les chemins d'images
image_paths = [os.path.join(images_dir, fname) for fname in os.listdir(images_dir) if fname.endswith('.jpg')]

# Fonction pour charger et prétraiter une image
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path.numpy().decode("utf-8"), target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return img_array

# Créez un Dataset d'images avec des formes définies
images_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
images_dataset = images_dataset.map(lambda x: tf.py_function(load_and_preprocess_image, [x], Tout=tf.float32))

# Fixer la forme explicitement après le chargement
images_dataset = images_dataset.map(lambda x: tf.ensure_shape(x, (224, 224, 3)))

# Associez les images avec leurs labels
labels = y_encoded
labels_tensor = tf.convert_to_tensor(labels)
labels_dataset = tf.data.Dataset.from_tensor_slices(labels_tensor)

# Combine les images et labels dans un Dataset unique
dataset = tf.data.Dataset.zip((images_dataset, labels_dataset)).batch(2)

# Modèle pré-entraîné VGG16 avec des couches personnalisées pour la classification
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Dégeler les dernières couches du modèle VGG16
base_model.trainable = True
for layer in base_model.layers[:-4]:  # Gèle toutes les couches sauf les 4 dernières
    layer.trainable = False

# Construire le modèle avec des couches de classification en sortie
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(7, activation="softmax")  # 7 classes
])

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraîner le modèle
model.fit(dataset, epochs=3)


Epoch 1/3
525/525 ━━━━━━━━━━━━━━━━━━━━ 30s 51ms/step - accuracy: 0.3665 - loss: 2.0748
Epoch 2/3
525/525 ━━━━━━━━━━━━━━━━━━━━ 33s 37ms/step - accuracy: 0.2595 - loss: 1.9293
Epoch 3/3
525/525 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.3478 - loss: 2.2813


In [34]:
# Affiche le résumé complet du modèle avec les formes de sortie
model.summary()


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 48,147,287 (183.67 MB)

 Trainable params: 13,504,007 (51.51 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

 Optimizer params: 27,008,016 (103.03 MB)

In [35]:
classifier.summary()

Preprocessor: "distil_bert_text_classifier_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)                   │                       Vocab size: 30,522 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "distil_bert_text_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 7)                 │           5,383 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 200,876,567 (766.28 MB)

 Trainable params: 66,958,855 (255.43 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 133,917,712 (510.86 MB)

In [43]:
from tensorflow.keras import Input, layers, Model
from tensorflow.keras.optimizers import Adam

# Définir les entrées explicitement
image_input = Input(shape=(224, 224, 3))
text_input = classifier.input  # On utilise directement l'entrée de `classifier`

# Obtenir les sorties de `model` pour les images
x_image = model(image_input)
x_image_output = x_image  # On prend la sortie dense déjà sans softmax

# Obtenir les sorties de `classifier` pour le texte
x_text_output = classifier.get_layer('pooled_dense').output  # Prendre la dernière couche dense

# Combinaison des deux sorties
combined = layers.Concatenate()([x_image_output, x_text_output])

# Ajout des couches de sortie pour la classification finale
x = layers.Dense(128, activation="relu")(combined)
x = layers.Dropout(0.5)(x)
output = layers.Dense(7, activation="softmax")(x)

# Création du modèle multi-input
multi_input_model = Model(inputs=[image_input, text_input], outputs=output)

# Compilation du modèle
multi_input_model.compile(optimizer=Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Affichage du résumé
multi_input_model.summary()


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ distil_bert_backbone      │ (None, None, 768)      │     66,362,880 │ padding_mask[0][0],    │
│ (DistilBertBackbone)      │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_24            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 768)            │              0 │ distil_bert_backbone[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_11             │ (None, 7)              │     21,139,271 │ input_layer_24[0][0]   │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pooled_dense (Dense)      │ (None, 768)            │        590,592 │ get_item[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 775)            │              0 │ sequential_11[0][0],   │
│ (Concatenate)             │                        │                │ pooled_dense[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_26 (Dense)          │ (None, 128)            │         99,328 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_19 (Dropout)      │ (None, 128)            │              0 │ dense_26[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_27 (Dense)          │ (None, 7)              │            903 │ dropout_19[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 88,192,974 (336.43 MB)

 Trainable params: 80,557,710 (307.30 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [44]:
# 1. Extraire la colonne `description` de votre DataFrame pour les textes
texts = df['description'].tolist()

# 2. Utiliser le préprocesseur de texte de DistilBERT pour transformer `texts` en entrées compatibles
texts_preprocessed = preprocessor(texts)  # Produit un dictionnaire avec `token_ids` et `padding_mask`

# 3. Charger les images
# Si `images_dataset` est déjà préparé, utilisez `np.array(list(images_dataset.as_numpy_iterator()))` pour créer un tableau NumPy d'images
images = np.array(list(images_dataset.as_numpy_iterator()))  # Convertit le dataset en un tableau NumPy

# 4. Préparer les labels
labels = np.array(y_encoded)  # Assurez-vous que `y_encoded` contient les labels pour chaque échantillon

# 5. Appel de la fonction `fit` avec les données multi-input
history = multi_input_model.fit(
    x={'input_layer_24': images, 'token_ids': texts_preprocessed['token_ids'], 'padding_mask': texts_preprocessed['padding_mask']},
    y=labels,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)



Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_318', {'token_ids': 'token_ids', 'padding_mask': 'padding_mask'}]. Received: the structure of inputs={'input_layer_24': '*', 'token_ids': '*', 'padding_mask': '*'}
  warnings.warn(


27/27 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.1242 - loss: 2.0005 - val_accuracy: 0.1667 - val_loss: 1.9944
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 63s 613ms/step - accuracy: 0.3069 - loss: 1.7470 - val_accuracy: 0.1476 - val_loss: 1.9552
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 626ms/step - accuracy: 0.3727 - loss: 1.5074 - val_accuracy: 0.2476 - val_loss: 1.7543
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 625ms/step - accuracy: 0.4277 - loss: 1.3657 - val_accuracy: 0.4571 - val_loss: 1.5692
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 610ms/step - accuracy: 0.5251 - loss: 1.1389 - val_accuracy: 0.4524 - val_loss: 1.4977
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 638ms/step - accuracy: 0.6210 - loss: 0.9235 - val_accuracy: 0.5143 - val_loss: 1.3745
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 651ms/step - accuracy: 0.6368 - loss: 0.8814 - val_accuracy: 0.5762 - val_loss: 1.4291
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 646ms/step - accuracy: 0.8071 - loss: 0.4941 - val_accuracy: 0.5333 

In [45]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,main_category
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa...",Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",...",Baby Care
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa...",Home Furnishing


In [52]:
# Charger et prétraiter l'image
image_path = '/content/drive/MyDrive/multi input/multi_input/dataset/Images_224/55b85ea15a1536d46b7190ad6fff8ce7.jpg'

def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le lot

# Prétraitement de l'image
image_input = load_and_preprocess_image(image_path)

# Prétraitement de la description
description = df['description'][0]  # Description spécifique
texts_preprocessed = preprocessor([description])  # Le préprocesseur attend une liste de texte

# Récupérer `token_ids` et `padding_mask` pour l'entrée du modèle
text_token_ids = np.expand_dims(texts_preprocessed['token_ids'][0], axis=0)
text_padding_mask = np.expand_dims(texts_preprocessed['padding_mask'][0], axis=0)

# Faire la prédiction avec le modèle multi-input
prediction = multi_input_model.predict({
    'input_layer_24': image_input,
    'token_ids': text_token_ids,
    'padding_mask': text_padding_mask
})

# Afficher les probabilités de prédiction pour chaque classe
print("Probabilités de prédiction :", prediction)

# Afficher la classe prédite
predicted_class = np.argmax(prediction, axis=1)
print("Classe prédite :", predicted_class)

# Créez un dictionnaire de mappage entre les indices encodés et les noms de classe
class_names_mapping = dict(zip(labels, df['main_category']))

# Obtenez l'indice de la classe prédite
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Utilisez le dictionnaire pour obtenir le nom de la classe prédite
predicted_class_name = class_names_mapping.get(predicted_class_index, "Classe inconnue")
print("Classe prédite :", predicted_class_name)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Probabilités de prédiction : [[0.05162273 0.05529219 0.00739199 0.00265403 0.8756928  0.00607031
  0.00127594]]
Classe prédite : [4]
Classe prédite : Home Furnishing


In [47]:
df['main_category'].unique()


array(['Home Furnishing', 'Baby Care', 'Watches',
       'Home Decor & Festive Needs', 'Kitchen & Dining',
       'Beauty and Personal Care', 'Computers'], dtype=object)

In [51]:

# Charger et prétraiter l'image
image_path = '/content/drive/MyDrive/multi input/multi_input/dataset/Images_224/7b72c92c2f6c40268628ec5f14c6d590.jpg'


def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le lot

# Prétraitement de l'image
image_input = load_and_preprocess_image(image_path)

# Prétraitement de la description
description = df['description'][1]  # Description spécifique
texts_preprocessed = preprocessor([description])  # Le préprocesseur attend une liste de texte

# Récupérer `token_ids` et `padding_mask` pour l'entrée du modèle
text_token_ids = np.expand_dims(texts_preprocessed['token_ids'][0], axis=0)
text_padding_mask = np.expand_dims(texts_preprocessed['padding_mask'][0], axis=0)

# Faire la prédiction avec le modèle multi-input
prediction = multi_input_model.predict({
    'input_layer_24': image_input,
    'token_ids': text_token_ids,
    'padding_mask': text_padding_mask
})

# Afficher les probabilités de prédiction pour chaque classe
print("Probabilités de prédiction :", prediction)

# Afficher la classe prédite
predicted_class = np.argmax(prediction, axis=1)
print("Classe prédite :", predicted_class)

# Créez un dictionnaire de mappage entre les indices encodés et les noms de classe
class_names_mapping = dict(zip(labels, df['main_category']))

# Obtenez l'indice de la classe prédite
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Utilisez le dictionnaire pour obtenir le nom de la classe prédite
predicted_class_name = class_names_mapping.get(predicted_class_index, "Classe inconnue")
print("Classe prédite :", predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Probabilités de prédiction : [[0.97040987 0.00946217 0.00193915 0.00324524 0.00631431 0.00753737
  0.00109192]]
Classe prédite : [0]
Classe prédite : Baby Care


In [53]:

# Charger et prétraiter l'image
image_path = '/content/drive/MyDrive/multi input/multi_input/dataset/montre.jpg'


def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le lot

# Prétraitement de l'image
image_input = load_and_preprocess_image(image_path)

# Prétraitement de la description
description = "Elegant Men's Chronograph Watch with Stainless Steel Mesh Band – This sophisticated timepiece features a sleek black dial with contrasting rose-gold markers and hands. Equipped with three sub-dials for seconds, minutes, and hours, this watch combines functionality with style. The durable stainless steel case and mesh band provide a modern, professional look, while the adjustable strap ensures a comfortable fit. With its quartz movement and date display, this watch is perfect for both casual and formal wear, adding a touch of refinement to any outfit."  # Description spécifique
texts_preprocessed = preprocessor([description])  # Le préprocesseur attend une liste de texte

# Récupérer `token_ids` et `padding_mask` pour l'entrée du modèle
text_token_ids = np.expand_dims(texts_preprocessed['token_ids'][0], axis=0)
text_padding_mask = np.expand_dims(texts_preprocessed['padding_mask'][0], axis=0)

# Faire la prédiction avec le modèle multi-input
prediction = multi_input_model.predict({
    'input_layer_24': image_input,
    'token_ids': text_token_ids,
    'padding_mask': text_padding_mask
})

# Afficher les probabilités de prédiction pour chaque classe
print("Probabilités de prédiction :", prediction)

# Afficher la classe prédite
predicted_class = np.argmax(prediction, axis=1)
print("Classe prédite :", predicted_class)

# Créez un dictionnaire de mappage entre les indices encodés et les noms de classe
class_names_mapping = dict(zip(labels, df['main_category']))

# Obtenez l'indice de la classe prédite
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Utilisez le dictionnaire pour obtenir le nom de la classe prédite
predicted_class_name = class_names_mapping.get(predicted_class_index, "Classe inconnue")
print("Classe prédite :", predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Probabilités de prédiction : [[2.6671204e-04 1.5820275e-04 5.3361210e-04 1.6607295e-04 5.0221907e-04
  6.0108076e-05 9.9831307e-01]]
Classe prédite : [6]
Classe prédite : Watches
